In [1]:
#######1
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
from nlpia.loaders import get_data
import glob
import os
import numpy as np
import io
from random import shuffle
def pre_process_data(filepath):
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    pos_label = 1
    neg_label = 0
    dataset = []
    
    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with io.open(filename, 'r', encoding='utf-8') as f:
            dataset.append((pos_label, f.read()))
    
    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with io.open(filename, 'r', encoding='utf-8') as f:       
            dataset.append((neg_label, f.read()))
    
    shuffle(dataset)
    
    return dataset


C:\ProgramData\Anaconda3\lib\site-packages\pugnlp\constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
C:\ProgramData\Anaconda3\lib\site-packages\pugnlp\constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
C:\ProgramData\Anaconda3\lib\site-packages\pugnlp\tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
C:\ProgramData\Anaconda3\lib\site-packages\pugnlp\util.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
INFO:nlpia.constants:Startin

In [2]:
def tokenize_and_vectorize3(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            
            except KeyError:
                pass # net slova
            
        vectorized_data.append(sample_vecs)
    
    return vectorized_data

In [3]:
def collect_expected(dataset):
    #""" Выбираем целевые значения из набора данных """
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

In [4]:
word_vectors = get_data('w2v', limit = 200000)

INFO:nlpia.loaders:Downloading w2v
INFO:nlpia.web:URL too short: w2v
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.300d\.zip$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.27b\.zip$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.42b\.zip$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.6b\.zip$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.bin\.gz$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.tgz$, string=googlenews-vectors-negative300.bin.gz
INFO:nlpia.loaders:expanded+normalized file path: C:\ProgramData\Anaconda3\lib\site-packages\nlpia\bigdata\googlenews-vectors-negative300.bin.gz
INFO:nlpia.loaders:requesting URL: https://www.dropbox.com/s/965dir4dje0hfi4/G

In [5]:
def pad_trunc(data, maxlen):
    new_data = []
    
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
        
    for sample in data:
        if(len(sample)) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
dataset = pre_process_data(r'C:\aclImdb_v1\aclImdb\train')
vectorized_data = tokenize_and_vectorize3(dataset)
expected = collect_expected(dataset)
split_point = int(len(vectorized_data) * .8)
x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]
maxlen = 400
batch_size = 32
embedding_dims = 300
epochs = 2
x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)
x_train = np.reshape(x_train,
                    (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)
num_neurons = 50
model = Sequential()
model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

model_structure = model.to_json()
with open("lstm_model1.json", "w") as json_file:
    json_file.write(model_structure)
    model.save_weights("lstm_weights1.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 400, 50)           70200     
_________________________________________________________________
dropout (Dropout)            (None, 400, 50)           0         
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 20001     
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
625/625 [==============================] - 93s 143ms/step - loss: 0.4694 - accuracy: 0.7831 - val_loss: 0.3645 - val_accuracy: 0.8508
Epoch 2/2
625/625 [==============================] - 73s 116ms/step - loss: 0.3403 - accuracy: 0.8572 - va

In [7]:
from keras.models import model_from_json
with open("lstm_model1.json", "r") as json_file:
    json_string = json_file.read()
    model = model_from_json(json_string)
    model.load_weights('lstm_weights1.h5')

In [8]:
sample_1 = """I hate niggers."""

In [9]:
vec_list = tokenize_and_vectorize3([(1, sample_1)])

In [10]:
test_vec_list = pad_trunc(vec_list, maxlen)

In [11]:
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

In [12]:
y_pred = model.predict(test_vec)
#y_pred = np.round(y_pred).astype(int)

In [13]:
print("Sample's sentiement, 1 - pos, 2 - neg : {}"\
      .format(y_pred))

Sample's sentiement, 1 - pos, 2 - neg : [[0.41108355]]


In [14]:
def zaebis_li_text(sample):
    vec_list = tokenize_and_vectorize3([(1, sample)])
    test_vec_list = pad_trunc(vec_list, maxlen)
    test_vec = np.reshape(test_vec_list, (len(test_vec_list), 
                                          maxlen, embedding_dims))
    y_pred = model.predict(test_vec)
    nascolko_norm = format(y_pred[0][0], '.2f') 
    print("Текст позитивный? 0 - нет, 1 - да : ", nascolko_norm)

    return y_pred 

In [15]:
y_pred_2 = zaebis_li_text("I hate this disgusting fucking shit")
y_pred_2 = zaebis_li_text("I love this beautiful amazing")

Текст позитивный? 0 - нет, 1 - да :  0.10
Текст позитивный? 0 - нет, 1 - да :  0.96


In [16]:
y_pred_2 = zaebis_li_text("Hitler did nothing wrong")

Текст позитивный? 0 - нет, 1 - да :  0.36
